In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [ ]:
#converting to tensor (standardizing using normalize)(0.1307-mean,0.3081-std dev)
transform =transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307),(0.3081))])

In [ ]:

kmnist_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Take the first sample from the dataset
sample_data, _ = kmnist_dataset[0]

# Get the shape of the sample data
shape_of_data = sample_data.shape

print("Shape of K-MNIST data sample:", shape_of_data)


In [ ]:
#load the train data
train_loader=torch.utils.data.DataLoader(kmnist_dataset,batch_size=64,shuffle=True)

In [ ]:
class Neural(nn.Module):
  def __init__(self):

    super(Neural,self).__init__()
    self.fc1=nn.Linear(3*32*32,256)
    self.fc2=nn.Linear(256,128)
    self.fc3=nn.Linear(128,64)
    self.fc4 = nn.Linear(64,32)
    self.fc5 = nn.Linear(32,16)
    self.fc6= nn.Linear(16,10)

  def forward(self,x):
     x = x.view(-1,3*32*32)
     x = torch.relu(self.fc1(x))
     x = torch.relu(self.fc2(x))
     x = torch.relu(self.fc3(x))
     x = torch.relu(self.fc4(x))
     x = torch.relu(self.fc5(x))
   # Applying activation for the new layer
     x = self.fc6(x)
     return x


net=Neural()

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.01,momentum=0.5)

In [ ]:
#Train the model

num_epochs=10
for epoch in range(num_epochs):
  for batch_idx,(data,target) in enumerate (train_loader):
    optimizer.zero_grad()
    output=net(data.view(-1,3*32*32))
    loss =criterion(output,target)
    loss.backward()
    optimizer.step()
    if batch_idx % 100==0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
#Load the test data and test the model

test_dataset=datasets.CIFAR10('data',train=False,download=True,transform=transform)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=1000,shuffle=True)
correct=0
total=0
with torch.no_grad():
  output=net(data)
  _,predicted=torch.max(output.data,1)
  total+=target.size(0)
  correct+=(predicted==target).sum().item()

In [ ]:

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))